In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Objective: To find inhibitors of Beta-Lactamase through ML/DL.

My approach is based on similarity based virtual screening. The entire workflow is as follows:

1. Search for already known inhibitors in Chembl database returned 19 molecules.
2. Based on literature,Tanimoto similarity coefficient with rdk fingerprints was calculated for all the 19 molecules against all molecules in the dataset.
3. A similarity threshold of 0.6 was set, which resulted in 319 molecules. All these 319 molecules were categorised as 'Active' and rest all as 'Inactive'.
4. Molecular descriptors (208) were calculated for the entire dataset and were used as features for machine learning.
5. A logistic regression model with regularization was fit along with ten folds cross-validation.
6. PCA analysis was done for dimensionality reduction, followed by logistic regression and cross-validation.

Result:
Logistic regression model with 10 folds cross-validation resulted in a mean AUC score of 1.00 and mean F1-score of 0.79.
PCA followed by logistic regression and 10 folds cross-validation resulted in a mean AUC score of 1.00 and mean F1-score of 0.73.


In [ ]:
# Reading the chembl dataset obtained from dataprofessor

df = pd.read_csv("../input/chembl-df/chembl_df.csv")
print(df.shape)
df.head(2)

#  Exploratory Data Analysis

In [ ]:
# Check for null values
import seaborn as sns
sns.heatmap(df.isnull(),cmap='viridis')

In [ ]:
# Dropping NA's
df.dropna(axis = 0,inplace = True)
df.shape

In [ ]:
import matplotlib.pyplot as plt
val = df.target_pref_name.value_counts()[:2]
fig = plt.figure(figsize = (3,3))
plt.pie(val,labels = val.keys(),autopct ='%1.1f%%',shadow = True,startangle = 90)
plt.show()

In [ ]:
val = df.standard_type.value_counts()[:3]
fig = plt.figure(figsize = (3,3))
plt.pie(val,labels = val.keys(),autopct = '%1.1f%%',shadow = True,startangle = 90)
plt.show()

In [ ]:
# Target of beta lactamse ampc and standard type of potency contributes to most of the data, hence they are selected for the analysis
amp_df = df[df.target_pref_name == 'Beta-lactamase AmpC']
amp_df = amp_df[amp_df.standard_type == 'Potency']

# Where duplicates are present,keep only the first one
amp_df.drop_duplicates(subset = 'molecule_chembl_id',keep = 'first',inplace = True)

# Final dataframe with desired columns
amp_df = amp_df[['molecule_chembl_id','canonical_smiles','pchembl_value']]
amp_df.head(2)

In [ ]:
!conda install -c rdkit rdkit -y

# **Similarity Based Virtual Screening**

In [ ]:
# Commenting to save commit time
# Tanimoto similarity coefficient was calculated with rdk fingerprints for all known 19 betalactamase inhibitors 
#found in chembl database against all molecules of dataset
# Similarity threshold was set to 0.6

from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import RDKFingerprint

# Checking similarity of sulbactam with other molecules in dataset
#sulbac = Chem.RDKFingerprint(Chem.MolFromSmiles('O=C(O)[C@H]1/C(=C/CO)O[C@@H]2CC(=O)N21'))
#smiles = {'ID': amp_df.molecule_chembl_id,'pchembl_value': amp_df.pchembl_value,'SMILES': amp_df.canonical_smiles }
#df = pd.DataFrame(data = smiles)
#df['Tanimoto_rdk'] = DataStructs.BulkTanimotoSimilarity(sulbac, [Chem.RDKFingerprint(Chem.MolFromSmiles(smi)) for smi in df['SMILES']])
#df['Dice_rdk'] = DataStructs.BulkDiceSimilarity(sulbac, [Chem.RDKFingerprint(Chem.MolFromSmiles(smi)) for smi in df['SMILES']])
#df=df[df.Tanimoto_rdk > 0.6]

# Similarity threshold file was saved and is read below

In [ ]:
# Duplicates were dropped

df2 = pd.read_csv('../input/rdk-sim/df2_rdk_sim.csv')
dup = df2[df2.SMILES.duplicated()]
print('Length of duplicates is:', len(dup))

df2.drop_duplicates(subset='SMILES',keep='first',inplace=True)
print('After dropping duplicates, length is',df2.shape[0])

In [ ]:
# Label encoding the target variable
# Active class is 1 and Inactive class is 0

smi = []
for i in df2.SMILES:
    smi.append(i)
    
amp_df['Class'] = amp_df['canonical_smiles'].apply(lambda x: any([k in x for k in smi]))

from sklearn.preprocessing import LabelEncoder
lab = LabelEncoder()
amp_df['Class'] = lab.fit_transform(amp_df.Class)
print(amp_df.Class.value_counts())

In [ ]:
# Molecular Descriptors of RDKIT

from rdkit.Chem import Descriptors   
from rdkit.ML.Descriptors import MoleculeDescriptors

descriptors_list = [x[0] for x in Descriptors._descList]
print(len(descriptors_list))
print(descriptors_list)

In [ ]:
# Commenting to save commit time
# Molecular descriptors have been calculated from rdkit.

#calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
#desc = []

#for smiles in amp_df.SMILES:
    #mol = Chem.MolFromSmiles(smiles)
    #descriptors = calc.CalcDescriptors(mol)
    #desc.append(descriptors)

#desc_df = pd.DataFrame(desc,columns=descriptors_list)

# Descriptor file was saved and read below

In [ ]:
desc_df = pd.read_csv('../input/descriptors/desc_df.csv')
desc_df.drop('Unnamed: 0',axis=1,inplace=True)
desc_df.head(2)

In [ ]:
# Logistic Regression with regularization

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x = desc_df.iloc[:,0:208]
x = x.fillna(x.mean())
x = StandardScaler().fit_transform(x)
y = amp_df.Class

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score,roc_auc_score,matthews_corrcoef,f1_score

model = LogisticRegression(solver='liblinear', C=10.0, random_state=0)
model.fit(x_train,y_train)
pred=model.predict(x_test)

print('AUC score is {:.2f}'.format(roc_auc_score(y_test,pred)))
print('Precision score is {:.2f}'.format(precision_score(y_test,pred)))
print('Matthews correlation coefficient is {:.2f}'.format(matthews_corrcoef(y_test,pred)))
print('F1 score is {:.2f}'.format(f1_score(y_test,pred)))

# Feature Importance

In [ ]:
(pd.Series(model.coef_[0], index=desc_df.columns)
   .nlargest(5)
   .plot(kind='barh')) 

# **Cross-Validation**

In [ ]:
# Logistic Regression with 10 folds cross-validation

from sklearn.model_selection import cross_val_score

x = desc_df.iloc[:,0:208]
x = x.fillna(x.mean())
x = StandardScaler().fit_transform(x)
y = amp_df.Class
model = LogisticRegression(solver='liblinear', C=10.0, random_state=0)

score_roc_auc = cross_val_score(model,x,y,cv=10,scoring='roc_auc')
print('The mean AUC score is {:.2f}'.format(score_roc_auc.mean()))

score_f1 = cross_val_score(model,x,y,cv=10,scoring='f1')
print('The mean F1 score is {:.2f}'.format(score_f1.mean()))

# **PCA Analysis**

In [ ]:
# PCA Analysis

from sklearn.decomposition import PCA

x = desc_df.iloc[:,0:208]
x=x.fillna(x.mean())
x = StandardScaler().fit_transform(x)
y = amp_df.Class

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

pca = PCA(0.95)
pca.fit(x_train)
print('The number of dimensions needed to achieve 95% variance is',pca.n_components_)


x_train = pca.transform(x_train)
x_test = pca.transform(x_test)

model = LogisticRegression(solver='liblinear', C=10.0, random_state=0)
model.fit(x_train,y_train)
pred = model.predict(x_test)

print('AUC score is {:.2f}'.format(roc_auc_score(y_test,pred)))
print('Precision score is {:.2f}'.format(precision_score(y_test,pred)))
print('Matthews correlation coefficient is {:.2f}'.format(matthews_corrcoef(y_test,pred)))
print('F1 score is {:.2f}'.format(f1_score(y_test,pred)))

# PCA Analysis with Cross-Validation

In [ ]:
x = desc_df.iloc[:,0:208]
x=x.fillna(x.mean())
x = StandardScaler().fit_transform(x)
y = amp_df.Class

pca = PCA(0.95)
pca.fit(x)
print('The number of dimensions needed to achieve 95% variance is',pca.n_components_)
x = pca.transform(x)

model = LogisticRegression(solver='liblinear', C=10.0, random_state=0)

score_roc_auc = cross_val_score(model,x,y,cv=10,scoring='roc_auc')
print('The mean AUC score is {:.2f}'.format(score_roc_auc.mean()))

score_f1 = cross_val_score(model,x,y,cv=10,scoring='f1')
print('The mean F1 score is {:.2f}'.format(score_f1.mean()))